In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))
from openTable import *
from AccesDB import *

from statistics import mean
from datetime import date
import pandas as pd
import numpy as np

def lolos(num, minimum, minimum_atas):
    if num >= minimum_atas:
        return "Kemungkinan Lolos Besar"
    elif num < minimum_atas and num >= minimum:
        return "Kemungkinan Lolos Sedang"
    else:
        return "Kemungkinan Lolos Kecil"

def get_data_jurusan():
    #DS
    ds_server,ds_koneksi = Connection_2()

    col = ['jurusanId', 'jenis', 'tahun_lalu_rerata', 'tahun_lalu_sbaku', 'tahun_lalu_kuota', 'tahun_lalu_peminat']
    ptn, status = open_table_ds(ds_koneksi,col, 'rasionalisasi_kampus')
    ptn = pd.DataFrame(ptn)
    ptn = ptn.rename(columns={0:'jurusanId', 1:'jenis', 2:'tahun_lalu_rerata', 
                                    3:'tahun_lalu_sbaku',4:'tahun_lalu_kuota', 5:'tahun_lalu_peminat'})
    
    #Get Data Pilihan
    data,status = open_table_ds(ds_koneksi,['userId','jurusanId'],'rasionalisasi_pilihan')
    data = pd.DataFrame(data)
    data = data.rename(columns={0:'userId',1:'jurusanId'})
    
    ds_koneksi.close()
    ds_server.stop()
    
    #Make Dictionary
    result ={}
    for i,j in ptn.iterrows():
        result[int(j['jurusanId'])] = {} 
        result[int(j['jurusanId'])]['jenis'] = j.jenis
        result[int(j['jurusanId'])]['daya_tampung'] = j.tahun_lalu_kuota
        result[int(j['jurusanId'])]['peminat'] = j.tahun_lalu_peminat
        result[int(j['jurusanId'])]['rerata'] = j.tahun_lalu_rerata
        result[int(j['jurusanId'])]['s_baku'] = j.tahun_lalu_sbaku

    groups = data.groupby(['jurusanId','userId'])['jurusanId'].agg('count')
    d = {k:list(groups[k].index) for k in groups.index.levels[0]}
    
    #Join Dictionary
    for k in result.keys():
        for l in d.keys():
            if k==l:
                result[k]['userId'] = d[l]
    
    return result

def get_data_pendaftar(): 
    #DS
    ds_server,ds_koneksi = Connection_2()
    
    data,status = open_table_ds(ds_koneksi,['*'],'rasionalisasi_nilai')
    data = pd.DataFrame(data)
    data = data.rename(columns={0:'userId',1:'jenis',2:'penalaran',3:'kuantitatif',4:'umum',5:'baca_tulis',6:'mat_saintek',7:'fisika',
                                8:'kimia',9:'biologi',10:'mat_soshum',11:'geografi',12:'sejarah',13:'sosiologi',14:'ekonomi',15:'rerata'})
    
    ds_koneksi.close()
    ds_server.stop()
    
    return data

def rerata_jurusan(dict_jurusan, data_pendaftar):
    saintek = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_saintek', 'fisika', 'kimia', 'biologi']
    soshum = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_soshum', 'geografi', 'sejarah', 'sosiologi', 'ekonomi']

    for k in dict_jurusan.keys():
        try:
            user = dict_jurusan[k]['userId']
            data = data_pendaftar.loc[data_pendaftar.userId.isin(user)]
            
            if dict_jurusan[k]['jenis'].lower() == 'saintek':
                for i in saintek:
                    dict_jurusan[k][i] = mean(list(data[i]))

            elif dict_jurusan[k]['jenis'].lower() == 'soshum':
                for i in soshum:
                    dict_jurusan[k][i] = mean(list(data[i]))

            dict_jurusan[k]['pendaftar'] = len(data)            
        except:
            pass
        
    return dict_jurusan

def logic(dict_jurusan,data_pendaftar):
    hasil_list = []
    rerata_list = []
    
    for k in dict_jurusan:
        try:
            kuota_atas = int(dict_jurusan[k]['daya_tampung']*0.2)
            user = dict_jurusan[k]['userId']
            data = data_pendaftar.loc[data_pendaftar.userId.isin(user)]
            data = data.sort_values('rerata',ascending=False)
            if dict_jurusan[k]['peminat'] > dict_jurusan[k]['daya_tampung']:
                generate = np.random.normal(dict_jurusan[k]['rerata'], dict_jurusan[k]['s_baku'], dict_jurusan[k]['daya_tampung'])

                gen = pd.DataFrame(generate, columns = ['rerata'])
                sbm = data.append(gen)

                minimum = sbm.nlargest(dict_jurusan[k]['daya_tampung'], 'rerata').tail(1)['rerata'].values[0]
                minimum_atas = sbm.nlargest(kuota_atas, 'rerata').tail(1)['rerata'].values[0]
            else:
                sbm = data

                minimum = 600 #batas bawah
                minimum_atas = 750 #threshold

            sbm = sbm.reset_index(drop=True)
            hasil_n = []
            
            for index, row in sbm.iterrows():
                hasil_n.append(lolos(row['rerata'], minimum, minimum_atas))

            sbm['hasil'] = hasil_n
            sbm = sbm[np.isfinite(sbm['userId'])]
            sbm['userId'] = sbm['userId'].astype('int')

            for index, row in sbm.iterrows():
                result = {}
                result['userId'] = row['userId']
                result['hasil'] = row['hasil']
                result['ranking'] = index+1
                result['jurusanId'] = k
                hasil_list.append(result)
                
        except Exception as e:
            pass
            
    return hasil_list

def save_db(data,dict_jurusan):
    #DS
    ds_server,ds_koneksi = Connection_2()

    today = date.today()
    # To Tabel pilihan
    for i in data:
        userId = i['userId']
        hasil = i['hasil']
        ranking = i['ranking']
        jurusanId = i['jurusanId']
        
        data_save = {
            'ranking':ranking,
            'tanggal':today,
            'recommendation':hasil
        }
        
        statement = ' where userId = {userId} and jurusanId = {jurusanId}'
        status = update_db(ds_koneksi,'rasionalisasi_pilihan',data_save,statement.format(userId=userId,jurusanId=jurusanId))
    
    # To Tabel hardcode
    for i in dict_jurusan:
        try:
            rerata = dict_jurusan[i]['rerata']
            r_penalaran = dict_jurusan[i]['penalaran']
            r_kuantitatif = dict_jurusan[i]['kuantitatif']
            r_umum = dict_jurusan[i]['umum']
            r_baca_tulis = dict_jurusan[i]['baca_tulis']
            pendaftar = dict_jurusan[i]['pendaftar']

            #IPA
            if dict_jurusan[i]['jenis'].lower()=='saintek':
                r_mat_saintek = dict_jurusan[i]['mat_saintek']
                r_fisika = dict_jurusan[i]['fisika']
                r_kimia = dict_jurusan[i]['kimia']
                r_biologi = dict_jurusan[i]['biologi']

                data_save = {
                    'tahun_ini_rerata':rerata,
                    'tanggal':today,
                    'tahun_ini_pendaftar':pendaftar,
                    'rerata_penalaran' :r_penalaran,
                    'rerata_kuantitatif':r_kuantitatif,
                    'rerata_umum':r_umum,
                    'rerata_baca_tulis':r_baca_tulis,
                    'rerata_mat_saintek':r_mat_saintek,
                    'rerata_fisika':r_fisika,
                    'rerata_kimia':r_kimia,
                    'rerata_biologi':r_biologi
                }

            #IPS
            elif dict_jurusan[i]['jenis'].lower()=='soshum':
                r_mat_soshum = dict_jurusan[i]['mat_soshum']
                r_sejarah = dict_jurusan[i]['sejarah']
                r_sosiologi = dict_jurusan[i]['sosiologi']
                r_ekonomi = dict_jurusan[i]['ekonomi']
                r_geografi = dict_jurusan[i]['geografi']

                data_save = {
                    'tahun_ini_rerata':rerata,
                    'tanggal':today,
                    'tahun_ini_pendaftar':pendaftar,
                    'rerata_penalaran' :r_penalaran,
                    'rerata_kuantitatif':r_kuantitatif,
                    'rerata_umum':r_umum,
                    'rerata_baca_tulis':r_baca_tulis,
                    'rerata_mat_soshum':r_mat_soshum,
                    'rerata_sejarah':r_sejarah,
                    'rerata_sosiologi':r_sosiologi,
                    'rerata_ekonomi':r_ekonomi,
                    'rerata_geografi':r_geografi
                }
                
            statement = ' where jurusanId = {jurusanId}'
            status = update_db(ds_koneksi,'rasionalisasi_kampus',data_save,statement.format(jurusanId=i))
            print(status)
        except:
            pass
    
    ds_koneksi.commit()
    ds_koneksi.close()
    ds_server.stop()
    
def batch():
    dict_jurusan = get_data_jurusan() #Get data univ
    data_pendaftar = get_data_pendaftar() #Get data pendatfar
    dict_jurusan = rerata_jurusan(dict_jurusan, data_pendaftar) #Get nilai rerata jurusan
    hasil = logic(dict_jurusan,data_pendaftar) #Get hasil
    save_db(hasil,dict_jurusan) #Save to DB

batch()

berhasil
berhasil
berhasil
berhasil
berhasil
